In [4]:
#Yu Yamaoka
#crop段階でフォルダPathの指定が関数内部で行ってしまっている点に注意．

#Function Define
from cellpose import models, io
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

# DEFINE CELLPOSE MODEL
# model_type='cyto' or model_type='nuclei'
def img_to_cellpose(img_path, model_type= 'cyto', chan=[0,0], min_size=40, gpu_enabled = True, model_path = ""):
    """
    Input:
        img_path : (string) Image file PATH
    Return:
        mask : [width, height]
    Args:
        model_type : https://github.com/MouseLand/cellpose/blob/main/cellpose/models.py#L19~L20
        chan : https://github.com/MouseLand/cellpose/blob/main/cellpose/models.py#L209
        min_size : https://github.com/MouseLand/cellpose/blob/main/cellpose/models.py#L175
        gpu_enabled : Are u install cuda?
        model_path : FineTuning model Path(most priority)
    """
    assert os.path.exists(img_path), ("image path is NOT exist")
    img = io.imread(img_path)
    
    # declare model
    if model_path != "":
        assert os.path.exists(model_path), ("model path is NOT exist")
        model = models.CellposeModel(gpu=gpu_enabled, pretrained_model=model_path)
        mask, _, _ = model.eval(img, diameter=None, channels=chan, min_size=min_size)
    else:
        model = models.Cellpose(gpu=gpu_enabled, model_type=model_type)
        mask, flows, styles, diams = model.eval(img, diameter=None, channels=chan, min_size=min_size)

    # save results so you can load in gui
    #io.masks_flows_to_seg(img, masks, flows, diams, img_path, chan)

    #save results as png
    #plt.imsave(img_path.replace(".png","CellposeMask.png"),masks)
    return mask

#mask画像をMaskRCNNが読み込めるデータセットにする。
def obj_detection(mask, class_id:int):
    """
    Input:
        mask : [width, height](ndarray), image data
        class_id : int , class id(ex : 1day -> 1)
    Return:
        mask : [width, height, n], n is object num.
        cls_idxs : [n(int)]
    """
    data = mask
    labels = []
    for label in np.unique(data):
        #: ラベルID==0は背景
        if label == 0:
            continue
        else:
            labels.append(label)

    if len(labels) == 0:
        #: 対象オブジェクトがない場合はNone
        return None, None
    else:
        mask = np.zeros((mask.shape)+(len(labels),), dtype=np.uint8)
        for n, label in enumerate(labels):
            mask[:, :, n] = np.uint8(data == label)
        cls_idxs = np.ones([mask.shape[-1]], dtype=np.int32) * class_id

        return mask, cls_idxs

def mask_to_patch(masks, img_path, size=64):
    """
    Input:
        masks : [n(objnum), width, height], n is object num.
        img_path : original image path of mask
    Return:
        crop_imgs :  [n(objnum), width, height, color]
        positions : [n, 4] 切り取った画像の座標[n, (height_min, height_max, width_min, width_max)]
    """
    #各mask-objのBBOXの重心点を求めて切り抜き
    img = cv2.imread(img_path)
    width, height, _ = img.shape
    crop_imgs = np.zeros((size, size, 3)+(len(masks),), dtype=np.uint8)
    positions = np.zeros((len(masks), 4), dtype=np.uint8)
    
    #1オブジェクトごとにcropをする
    for i in range(len(masks)):
        #重心計算
        mu = cv2.moments(masks[i], False)
        g_height, g_width = int(mu["m10"]/mu["m00"]) , int(mu["m01"]/mu["m00"])
        
        height_min = g_height - size//2
        height_max = g_height + size//2
        width_min = g_width - size//2
        width_max = g_width + size//2
        
        #壁際の例外処理
        if height_min<0:#配列外アクセスを防ぐ
            height_min = 0
            height_max= (size//2) * 2
        elif height_max>height:#配列外アクセスを防ぐ
            height_min = height - ((size//2)  * 2 + 1)
            height_max = height - 1
    
        #壁際の例外処理
        if width_min<0:#配列外アクセスを防ぐ
            width_min = 0
            width_max= (size//2) * 2
        elif width_max>width:#配列外アクセスを防ぐ
            width_min = width - ((size//2)  * 2 + 1)
            width_max = width - 1
        
        crop_imgs[:, :, :, i] = img[width_min:width_max, height_min:height_max]
        positions[i] = [height_min, height_max, width_min, width_max]

    crop_imgs = crop_imgs.transpose(3, 0 , 1, 2)
    return crop_imgs, positions



from turtle import onrelease
import cv2
import matplotlib.pyplot as plt
import numpy as np
import feret

def Compute_CellFeature(onemask):
    """
    Args:
        onemask : [width, height]
    Return:
        feature : [](area, length, circularity)
    """
    #白い領域の割合計算(0~1)
    w, h = onemask.shape
    #white_area = cv2.countNonZero(onemask)
    
    #Compute area, len, cir
    contours, _ = cv2.findContours(onemask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE) 
    area = cv2.contourArea(contours[0], True)
    length = cv2.arcLength(contours[0], True)

    try:
        circularity = 4*np.pi*area/length/length
    except:
        circularity = 0
        print(length)
        print("length is 0")
    
    #Compute feret
    maxf, minf, minf90, maxf90 = feret.all(onemask)
    feret_ratio = maxf/minf

    return [abs(area), abs(length), abs(circularity), maxf,minf, minf90, minf90, maxf90, feret_ratio]


In [6]:
from tqdm import tqdm
from glob import glob
import csv

folders = glob("./picture/*")

for folder in folders:
    cell = 0
    area = 0
    length = 0
    
    files = glob(folder + "/*.png")
    
    for file in tqdm(files):
        try:
            masks = img_to_cellpose(file)
            masks, ids = obj_detection(masks, 1)#return [width, height, N]
            masks = masks.transpose(2, 1, 0)#(x,y,obj番号)→(obj番号,x,y)に転置．
            
            cell += len(masks)
            for mask in masks:
                feature = Compute_CellFeature(mask)
                area += feature[0]
                length += feature[1]
        except:
            print(file)
    
    print("cell_num : " + str(cell))
    print("area_average : " + str(area/cell))
    print("length_ave : " + str(length/cell))
    
    filename = "data.csv"
    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([folder] + [cell, area/cell, length/cell])   
        

  1%|▏         | 2/136 [00:01<01:22,  1.62it/s]/a/yu-yamaoka/anaconda3/envs/cell/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/a/yu-yamaoka/anaconda3/envs/cell/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  2%|▏         | 3/136 [00:01<01:15,  1.77it/s]

./picture/0day_No3L_220424.tif.png/10.png


  3%|▎         | 4/136 [00:02<01:11,  1.84it/s]

./picture/0day_No3L_220424.tif.png/11.png


  7%|▋         | 10/136 [00:05<01:16,  1.64it/s]

...

 70%|███████   | 101/144 [01:06<00:26,  1.65it/s]

./picture/0day_No4L_220424.tif.png/100.png


 72%|███████▏  | 103/144 [01:08<00:26,  1.56it/s]

0.0
length is 0


 75%|███████▌  | 108/144 [01:12<00:26,  1.37it/s]